# Computer Vision I
## Motion Detection Using Simple Image Filtering

### Importing Dependencies

In [ ]:
import os
import cv2
import time
import numpy as np
import math
import scipy.signal as signal

#/Course Work Northeastern/ComputerVision/Project/Project1/Code
path = "Data/RedChair"
# path = "Data/Office"

dir_list = os.listdir(path)
  


### Function to generate 1D Derivative of Gaussian

In [ ]:
def Gauss_1D(tsigma):    
    """
    Inputs:
    - tsigma: The standard deviation of the Gaussian filter

    Outputs:
    - A 1-dimensional Gaussian filter of length 5*tsigma, with values normalized by the minimum value and divided by the 
    sum of the values.

    Steps:
    1. Compute the length of the filter.
    2. Generate the Gaussian filter using the `gaussian` function.
    3. Normalize the filter by dividing each value by the minimum value.
    4. Print the normalization factor and the values of the normalized filter.
    5. Return the normalized filter, divided by the sum of its values.
    
    """
    
    length = int(5 * tsigma)    
    gaussian = signal.gaussian(length, std=tsigma)
    min_y = min(gaussian)
    for i in range(length):
        gaussian[i] = np.round(gaussian[i]/min_y)
    print('(1/',sum(gaussian),')*',gaussian)
    return gaussian/sum(gaussian)


### Q1 Function to which calculates the differential and displays the masked image with threshold = 10

In [ ]:
def motion_det_10(folder='RedChair',filter_kernel = np.array([-1,0,1]),smoothening=False,derivative='derivative_filter'):
    """
    This is a Python function named motion_det that detects motion in a sequence of images given the directory path to the 
    images, a filter kernel for computing the temporal derivative, and a few other optional parameters.

    The function first sets the directory path to the images and lists all the images in the directory. If the smoothening 
    parameter is set to True, the function smoothes the images using either a box filter or a Gaussian filter based on the 
    smoothening parameter's value. If derivative is set to 'gauss', the function replaces the filter kernel with a 1D 
    Gaussian 
    filter.

    The function then applies the filter kernel to each pixel in the buffer of the sequence of images, computes the 
    absolute value of the temporal derivative, and threshold the absolute value to create a binary mask. It writes the 
    resultant motion detection output to a directory with a postfix of '_result' appended to the original directory. The 
    function returns three lists of values: mask_append which contains the masked images, abs_derivative_append which 
    contains the absolute derivatives of the images, and threshold_val_li which contains the threshold values used for 
    each image.
    
    """    

    
    folder = f'Data/{folder}'
    dir_list = os.listdir(f'{folder}')
    if smoothening:
            smoothening = smoothening.split('_')
            size = int(smoothening[1])
            if smoothening[0]=='gauss':
                ssigma = input('ssigma = ')       
            print(smoothening)
    if derivative=='gauss':
            tsigma = float(input('tsigma = '))
            filter_kernel = Gauss_1D(tsigma)
            print('kernel=',filter_kernel)
        
    abs_derivative_append = []
    mask_append = []
    buffer = []
    threshold_val_li = []
    
    threshold_value = 10 #default thresh value (assumed)
    
    for im in dir_list:
        img = cv2.imread(f'{folder}/{im}')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        if smoothening:
            if smoothening[0]=='box':
                gray = cv2.boxFilter(gray,ddepth=0,ksize=(size,size))
                
            elif smoothening[0]=='gauss':
                
                l = int(5*ssigma[0])
                gray = cv2.GaussianBlur(gray, (0,0), int(ssigma),int(ssigma))
                
        buffer.append(gray)

        if len(buffer) >= 3:
            # Applying the filter to each pixel in the buffer
            derivative_0 = cv2.filter2D(buffer[0], -1, filter_kernel)  # previous frame
            derivative_1 = cv2.filter2D(buffer[2], -1, filter_kernel)  # next frame    
                                
            # Compute the absolute value of the temporal derivative
            abs_derivative_x = cv2.absdiff(derivative_0, derivative_1)
            
            std_abs_der = np.std(abs_derivative_x)
            threshold_value = 10
                                
            
            threshold_val_li.append(threshold_value)
            # Threshold the absolute value to create a binary mask            
            _, mask = cv2.threshold(abs_derivative_x, threshold_value, 255, cv2.THRESH_BINARY)

            masked_image = np.multiply(mask,buffer[1])
            gray = np.hstack((buffer[1], masked_image,abs_derivative_x))
            
            abs_derivative_append.append(abs_derivative_x)
            mask_append.append(masked_image)
            try:
                cv2.imwrite(f'E:/Course Work Northeastern/ComputerVision/Project/Project1/Code/{folder}_result/{im}_res.jpg', gray)
            except Exception as ex:
                print(ex)
            
            buffer.pop(0)
            cv2.imshow(f'Gray',gray)
            cv2.imshow('img',img)
            k = cv2.waitKey(40) & 0xFF
            if k==27 or k==ord('q') or k==ord('Q'):
                cv2.destroyAllWindows()
                break

    cv2.destroyAllWindows()   
    return mask_append,abs_derivative_append,threshold_val_li

### Q2 is based on function calls mentioned at the end

### Q3 Function to which calculates the differential and displays the masked image with dynamic thesholding

In [ ]:
def motion_det(folder='RedChair',filter_kernel = np.array([-1,0,1]),smoothening=False,derivative='derivative_filter'):
    """
    This is a Python function named motion_det that detects motion in a sequence of images given the directory path to the 
    images, a filter kernel for computing the temporal derivative, and a few other optional parameters.

    The function first sets the directory path to the images and lists all the images in the directory. If the smoothening 
    parameter is set to True, the function smoothes the images using either a box filter or a Gaussian filter based on the 
    smoothening parameter's value. If derivative is set to 'gauss', the function replaces the filter kernel with a 1D 
    Gaussian 
    filter.

    The function then applies the filter kernel to each pixel in the buffer of the sequence of images, computes the 
    absolute value of the temporal derivative, and threshold the absolute value to create a binary mask. It writes the 
    resultant motion detection output to a directory with a postfix of '_result' appended to the original directory. The 
    function returns three lists of values: mask_append which contains the masked images, abs_derivative_append which 
    contains the absolute derivatives of the images, and threshold_val_li which contains the threshold values used for 
    each image.
    
    """    

    
    folder = f'Data/{folder}'
    dir_list = os.listdir(f'{folder}')
    if smoothening:
            smoothening = smoothening.split('_')
            size = int(smoothening[1])
            if smoothening[0]=='gauss':
                ssigma = input('ssigma = ')       
            print(smoothening)
    if derivative=='gauss':
            tsigma = float(input('tsigma = '))
            filter_kernel = Gauss_1D(tsigma)
            print('kernel=',filter_kernel)
        
    abs_derivative_append = []
    mask_append = []
    buffer = []
    threshold_val_li = []
    
    threshold_value = 10 #default thresh value (assumed)
    
    for im in dir_list:
        img = cv2.imread(f'{folder}/{im}')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        if smoothening:
            if smoothening[0]=='box':
                gray = cv2.boxFilter(gray,ddepth=0,ksize=(size,size))
                
            elif smoothening[0]=='gauss':
                
                l = int(5*ssigma[0])
                gray = cv2.GaussianBlur(gray, (0,0), int(ssigma),int(ssigma))
                
        buffer.append(gray)

        if len(buffer) >= 3:
            # Applying the filter to each pixel in the buffer
            derivative_0 = cv2.filter2D(buffer[0], -1, filter_kernel)  # previous frame
            derivative_1 = cv2.filter2D(buffer[2], -1, filter_kernel)  # next frame    
                                
            # Compute the absolute value of the temporal derivative
            abs_derivative_x = cv2.absdiff(derivative_0, derivative_1)
            
            std_abs_der = np.std(abs_derivative_x)
            temp_max_int = 0
            if derivative == 'derivative_filter':
                if std_abs_der<1.0: 
                    temp_max_int = abs_derivative_x.max()
                    threshold_value = temp_max_int*1.1
                else:
                    threshold_value = std_abs_der*5 #std_abs_der*3
                    
            else:
                if std_abs_der<1.0: #1.5:
                    temp_max_int = abs_derivative_x.max()
                    threshold_value = temp_max_int*1.1

                else:
                    threshold_value = cv2.THRESH_OTSU #+ 10
                                
            
            threshold_val_li.append(threshold_value)
            # Threshold the absolute value to create a binary mask            
            _, mask = cv2.threshold(abs_derivative_x, threshold_value, 255, cv2.THRESH_BINARY)

            masked_image = np.multiply(mask,buffer[1])
            gray = np.hstack((buffer[1], masked_image,abs_derivative_x))
            
            abs_derivative_append.append(abs_derivative_x)
            mask_append.append(masked_image)
            try:
                cv2.imwrite(f'E:/Course Work Northeastern/ComputerVision/Project/Project1/Code/{folder}_result/{im}_res.jpg', gray)
            except Exception as ex:
                print(ex)
            
            buffer.pop(0)
            cv2.imshow(f'Gray',gray)
            cv2.imshow('img',img)
            k = cv2.waitKey(40) & 0xFF
            if k==27 or k==ord('q') or k==ord('Q'):
                cv2.destroyAllWindows()
                break

    cv2.destroyAllWindows()   
    return mask_append,abs_derivative_append,threshold_val_li

### Function Call for "Office" Data


#### Derivative Filter Kernel : 1/2[-1,0,1]


In [ ]:
filter_kernel = np.array([-0.5,0,0.5])
folder = 'Office'
derivative = 'derivative_filter'
mask_append,abs_derivative_append,threshold_val_li = motion_det_10(folder=folder,derivative=derivative,smoothening = False,filter_kernel=filter_kernel)

#### Gauss filter with user defined tsigma

In [ ]:
folder = 'Office'
derivative = 'gauss'
mask_append,abs_derivative_append,threshold_val_li = motion_det(folder=folder,derivative=derivative,smoothening = False)

#### 2D spatial smoothing 

In [ ]:
filter_kernel = np.array([-0.5,0,0.5])
folder = 'Office'
smooth_filter = 'box_3'
# smooth_filter = 'box_5'
# smooth_filter = 'gauss_5'
# derivative = 'derivative_filter'
derivative = 'gauss'
mask_append,abs_derivative_append,threshold_val_li = motion_det_10(folder=folder,derivative=derivative,smoothening = smooth_filter)

### Function Call for "RedChair" Data


#### Derivative Filter Kernel : 1/2[-1,0,1]


In [ ]:
filter_kernel = np.array([-0.5,0,0.5])
folder = 'RedChair'
derivative = 'derivative_filter'
mask_append,abs_derivative_append,threshold_val_li = motion_det(folder=folder,derivative=derivative,smoothening = False,filter_kernel=filter_kernel)

#### Gauss filter with user defined tsigma

In [ ]:
folder = 'RedChair'
derivative = 'gauss'
mask_append,abs_derivative_append,threshold_val_li = motion_det(folder=folder,derivative=derivative,smoothening = False)

#### 2D spatial smoothing 

In [ ]:
filter_kernel = np.array([-0.5,0,0.5])
folder = 'RedChair'
smooth_filter = 'box_3'
# smooth_filter = 'box_5'
# smooth_filter = 'gauss_5'
derivative = 'derivative_filter'
# derivative = 'gauss'
mask_append,abs_derivative_append,threshold_val_li = motion_det_10(folder=folder,derivative=derivative,smoothening = smooth_filter)